In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [6]:

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(BinaryClassifier, self).__init__()
        layers = []
        #add fully connected layers with the input dim dividing by two each time the input shape
        output_dim = max(input_dim//(2), 1)
        for i in range(hidden_layers):
            i += 1
            layers.append(nn.Linear(input_dim,output_dim))
            layers.append(nn.ReLU())
            input_dim = output_dim
            output_dim = max(input_dim//(2*i), 1)
        #add final layer with sigmiod output and final linear layer
        layers.append(nn.Linear(input_dim,1))
        layers.append(nn.Sigmoid())

        #create a Seqeuntial implementation, no additional layers but provides one line to run through all layers
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        #apply the neural net
        return self.network(x)

In [8]:
#input shape is not correct
input_shape = 30
hidden_layers = 5
model_1 = BinaryClassifier(input_shape, hidden_layers)

In [14]:
optimizer = optim.Adam(model_1.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

In [16]:
#import whatever data loader function is needed
from cancer_train_test_split import train_test_split
#should output train dataloader and test dataloader  as the full train and test datasets
train_data , test_data = train_test_split()

ModuleNotFoundError: No module named 'cancer_train_test_split'

In [ ]:
#turn train data into epochs

In [ ]:
# Training loop
# def train(loss_fn,optimizer, data, model):
#     num_epochs = 10
#     epochs_loss = []
#     epochs_acr = []
#     for epoch in range(num_epochs):
#         model.train()
#         epoch_loss = 0.0
#         correct = 0
#         total = 0
        
#         for batch_X, batch_y in data:
#             optimizer.zero_grad()
            
#             outputs = model(batch_X)            # forward
#             loss = loss_fn(outputs, batch_y)  # compute loss
#             loss.backward()                     # backprop
#             optimizer.step()                    # update weights
            
#             epoch_loss += loss.item() * batch_X.size(0)
            
#             # accuracy calculation
#             preds = (outputs > 0.5).float()
#             correct += (preds == batch_y).sum().item()
#             total += batch_y.size(0)
        
#         avg_loss = epoch_loss / total
#         accuracy = correct / total
#         epochs_loss.append(avg_loss)
#         epochs_acr.append(accuracy)
#         print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Acc: {accuracy:.4f}")
#     return model, epochs_loss, epochs_acr

In [ ]:
# Compatibility check
print("=== COMPATIBILITY CHECK ===")
print(f"Input shape: {input_shape}")
print(f"Hidden layers: {hidden_layers}")
print(f"Train data type: {type(train_data)}")

# Test model creation
test_model = BinaryClassifier(input_shape, hidden_layers)
print(f"Model created successfully: {test_model}")

# Test with one batch
for batch_X, batch_y in train_data:
    print(f"Batch X shape: {batch_X.shape}")
    print(f"Batch y shape: {batch_y.shape}")
    print(f"Batch y values: {batch_y.unique()}")
    
    # Test forward pass
    test_output = test_model(batch_X)
    print(f"Model output shape: {test_output.shape}")
    print(f"Model output range: {test_output.min().item():.4f} to {test_output.max().item():.4f}")
    break

print("✅ All compatibility checks passed!")

In [ ]:
# Training loop

def validate_fold(model, val_loader, criterion, device='cpu'):
    """Validation function for each fold"""
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.float().unsqueeze(1))
            
            test_loss += loss.item() * batch_X.size(0)
            
            # Convert outputs to probabilities -> predictions
            preds = (outputs > 0.5).float()
            correct += (preds == batch_y.float().unsqueeze(1)).sum().item()
            total += batch_y.size(0)
    
    avg_loss = test_loss / total
    accuracy = correct / total
    
    return avg_loss, accuracy

def train_with_stratified_kfold(train_data, input_shape, hidden_layers, k_folds=5, num_epochs=10, device='cpuf'):
    """
    Train model using stratified k-fold cross-validation
    """
    print("Extracting data from DataLoader...")
    
    # Convert your train_data to numpy arrays
    X_list = []
    y_list = []
    
    # Extract features and labels from your DataLoader
    for batch_X, batch_y in train_data:
        X_list.append(batch_X.numpy())
        y_list.append(batch_y.numpy())
    
    X = np.vstack(X_list)
    y = np.hstack(y_list)
    
    print(f"Dataset shape: X={X.shape}, y={y.shape}")
    print(f"Class distribution: {np.bincount(y.astype(int))}")
    
    # Stratified K-Fold setup
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_results = []
    all_fold_losses = []
    
    print(f"\nStarting {k_folds}-Fold Stratified Cross Validation...")
    print("="*60)
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold + 1}/{k_folds} ---")
        
        # Split data for this fold
        X_train_fold = torch.FloatTensor(X[train_idx])
        X_val_fold = torch.FloatTensor(X[val_idx])
        y_train_fold = torch.LongTensor(y[train_idx])
        y_val_fold = torch.LongTensor(y[val_idx])
        
        print(f"Train size: {len(X_train_fold)}, Validation size: {len(X_val_fold)}")
        
        # Create datasets and dataloaders for this fold
        train_dataset_fold = torch.utils.data.TensorDataset(X_train_fold, y_train_fold)
        val_dataset_fold = torch.utils.data.TensorDataset(X_val_fold, y_val_fold)
        
        train_loader_fold = DataLoader(train_dataset_fold, batch_size=32, shuffle=True)
        val_loader_fold = DataLoader(val_dataset_fold, batch_size=32, shuffle=False)
        
        # Initialize fresh model for this fold
        model = BinaryClassifier(input_shape, hidden_layers)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCELoss()
        
        # Training variables for this fold
        epochs_loss = []
        epochs_acc = []
        best_val_acc = 0.0
        best_val_loss = float('inf')
        
        # Training loop for this fold
        for epoch in range(num_epochs):
            model.train()
            epoch_loss = 0.0
            correct = 0
            total = 0
            
            for batch_X, batch_y in train_loader_fold:
                optimizer.zero_grad()
                
                outputs = model(batch_X)
                loss = loss_fn(outputs, batch_y.float().unsqueeze(1))
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item() * batch_X.size(0)
                
                # Calculate accuracy
                preds = (outputs > 0.5).float()
                correct += (preds == batch_y.float().unsqueeze(1)).sum().item()
                total += batch_y.size(0)
            
            # Calculate average loss and accuracy for this epoch
            avg_loss = epoch_loss / total
            accuracy = correct / total
            epochs_loss.append(avg_loss)
            epochs_acc.append(accuracy)
            
            # Validation for this fold
            val_loss, val_acc = validate_fold(model, val_loader_fold, loss_fn)
            
            # Track best validation performance
            if val_acc > best_val_acc:
                best_val_acc = val_acc
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            
            if epoch % 2 == 0 or epoch == num_epochs - 1:  # Print every 2 epochs + last epoch
                print(f'Epoch [{epoch+1:2d}/{num_epochs}] '
                      f'Train Loss: {avg_loss:.4f} | Train Acc: {accuracy:.4f} | '
                      f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
        
        fold_results.append(best_val_acc)
        all_fold_losses.append(best_val_loss)
        print(f"Fold {fold + 1} Results - Best Val Acc: {best_val_acc:.4f}, Best Val Loss: {best_val_loss:.4f}")
    
    # Calculate overall results
    mean_acc = np.mean(fold_results)
    std_acc = np.std(fold_results)
    mean_loss = np.mean(all_fold_losses)
    std_loss = np.std(all_fold_losses)
    
    print(f"\n" + "="*60)
    print("STRATIFIED K-FOLD CROSS VALIDATION RESULTS")
    print("="*60)
    print(f"Individual Fold Accuracies: {[f'{acc:.4f}' for acc in fold_results]}")
    print(f"Individual Fold Losses:     {[f'{loss:.4f}' for loss in all_fold_losses]}")
    print(f"Mean Validation Accuracy:   {mean_acc:.4f} ± {std_acc:.4f}")
    print(f"Mean Validation Loss:       {mean_loss:.4f} ± {std_loss:.4f}")
    print("="*60)
    
    return fold_results, mean_acc, std_acc, all_fold_losses, mean_loss, std_loss

# MAIN EXECUTION - Replace your training loop with this:
print("Starting Stratified K-Fold Cross Validation Training...")

# Run stratified k-fold cross validation
fold_results, mean_acc, std_acc, fold_losses, mean_loss, std_loss = train_with_stratified_kfold(
    train_data,      # Your existing train_data DataLoader
    input_shape,     # Your existing input_shape variable
    hidden_layers,   # Your existing hidden_layers variable
    k_folds=5,       # Number of folds
    num_epochs=10    # Number of epochs per fold
    device='cuda' if torch.cuda.is_available() else 'cpu' #call gpu for processing, if not available use cpu
)

# Plot the results
plt.figure(figsize=(12, 5))

# Plot 1: K-Fold Accuracy Results
plt.subplot(1, 2, 1)
plt.bar(range(1, len(fold_results) + 1), fold_results, color='skyblue', alpha=0.7)
plt.axhline(y=mean_acc, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_acc:.3f}')
plt.fill_between(range(1, len(fold_results) + 1), 
                 mean_acc - std_acc, mean_acc + std_acc, 
                 alpha=0.2, color='red', label=f'±1 Std: {std_acc:.3f}')
plt.xlabel('Fold Number')
plt.ylabel('Validation Accuracy')
plt.title('Stratified K-Fold Cross Validation Results')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: K-Fold Loss Results  
plt.subplot(1, 2, 2)
plt.bar(range(1, len(fold_losses) + 1), fold_losses, color='lightcoral', alpha=0.7)
plt.axhline(y=mean_loss, color='darkred', linestyle='--', linewidth=2, label=f'Mean: {mean_loss:.3f}')
plt.xlabel('Fold Number')
plt.ylabel('Validation Loss')
plt.title('Stratified K-Fold Loss Results')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final summary
print(f"\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"✓ Completed {len(fold_results)}-fold stratified cross validation")
print(f"✓ Mean Validation Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
print(f"✓ Mean Validation Loss:     {mean_loss:.4f} ± {std_loss:.4f}")

if std_acc < 0.05:
    print("✓ EXCELLENT: Low standard deviation - model is very stable!")
elif std_acc < 0.10:
    print("✓ GOOD: Moderate standard deviation - model is reasonably stable")
else:
    print("⚠ CAUTION: High standard deviation - model performance varies significantly")

print("="*60)

In [ ]:
def test(model, data, criterion):
    model.eval()  # evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in data:
            
            logits = model(batch_X)                # raw outputs
            loss = criterion(logits, batch_y)
            
            test_loss += loss.item() * batch_X.size(0)
            
            # convert logits -> probabilities -> predictions
            preds = torch.sigmoid(logits) > 0.5
            correct += (preds.float() == batch_y).sum().item()
            total += batch_y.size(0)
    
    avg_loss = test_loss / total
    accuracy = correct / total
    print(f"Test Loss: {avg_loss:.4f} | Test Acc: {accuracy:.4f}")
    return avg_loss, accuracy

In [ ]:
#training model
trained_model_1,epochs_loss,epochs_arc = train(loss_fn,optimizer, train_epoch_data, model_1)

In [ ]:
#testing model
avg_loss, accuracy = test(trained_model, train_data, loss_fn)
print(f'testing loss {avg_loss}, model accuracy {accuracy}')

In [ ]:
plt.plot(epochs_loss)
plt.ylabel('training loss')
plt.xlabel('epochs')
plt.show()